In [ ]:
.
├── README.md
├── data
│   ├── bank_activity.csv           # Placeholder for the dataset
├── models
│   └── temporal_transformer.py     # Temporal Diffusion Transformer model
├── notebooks
│   └── anomaly_detection.ipynb     # Jupyter Notebook for data analysis and anomaly detection
├── src
│   ├── preprocessing.py            # Data preprocessing scripts
│   ├── train.py                    # Script to train the model
│   ├── evaluate.py                 # Script to evaluate and identify anomalies
└── requirements.txt                # Python dependencies


In [1]:
# preprocessing.py
import pandas as pd
from sklearn.preprocessing import StandardScaler

def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    # Fill missing values, normalize features
    df.fillna(method='ffill', inplace=True)
    scaler = StandardScaler()
    df[['debit_active_accounts']] = scaler.fit_transform(df[['debit_active_accounts']])
    return df


In [2]:
# temporal_transformer.py
import torch
import torch.nn as nn

class TemporalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers):
        super(TemporalTransformer, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, 1) # Predict debit active accounts

    def forward(self, x):
        x = self.transformer(x)
        output = self.fc(x)
        return output


In [5]:
import torch
import torch.nn as nn

class TemporalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers):
        super(TemporalTransformer, self).__init__()
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers
        )
        self.fc = nn.Linear(input_dim, 1)  # Example final layer

    def forward(self, x):
        # Assuming x has shape (batch_size, seq_length, input_dim)
        x = x.permute(1, 0, 2)  # Transformer expects (seq_length, batch_size, input_dim)
        x = self.transformer(x, x)
        x = x.permute(1, 0, 2)  # Back to (batch_size, seq_length, input_dim)
        x = self.fc(x[:, -1, :])  # Predict based on the last time step
        return x


In [6]:
import matplotlib.pyplot as plt

def plot_anomalies(df, anomalies):
    plt.plot(df['date'], df['debit_active_accounts'], label='Debit Active Accounts')
    plt.scatter(df['date'].iloc[anomalies], df['debit_active_accounts'].iloc[anomalies], color='red', label='Anomalies')
    plt.legend()
    plt.show()


In [9]:
# Install necessary packages
!pip install torch pandas scikit-learn matplotlib seaborn numpy

# Generate requirements.txt
!pip freeze > requirements.txt
